# Import bibs

In [ ]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns
sns.set()

# Load and inspect data set

In [ ]:
# Fetch the file
my_file = project.get_file(".csv")

# Read the CSV data file from the object storage into a pandas DataFrame
my_file.seek(0)
original_data = pd.read_csv(my_file)

original_data.head()

In [ ]:
original_data.describe(include='all') # descriptive statistics for all columns

In [ ]:
original_data.isnull().sum() # check for null values

In [ ]:
original_data[original_data.duplicated(keep=False)] # check for duplicate rows

Null values: daten können gedropt werden. Entweder Spalten (wenn sehr viele NaN) oder die Rows mit den NaNs, wenn es nicht so viele Rows betrifft.
Nochmal die Möglichkeiten mit Code:

In [ ]:
# a) drop the rows where a specific value is missing
df_wo_null = original_data.dropna(axis=0, subset=['attr'])

In [ ]:
# b) drop the whole column if too many values are missing
df_wo_null = df_wo_null.drop(['attr'], axis = 1)

In [ ]:
# c) replace missing values with the most frequent value
df_wo_null['attr'] = df_wo_null['attr'].fillna(df_wo_null['attr'].mode().iloc[0])

#### Inspect features

#### Correlation Map

In [ ]:
f,ax=plt.subplots(figsize = (18,18))
sns.heatmap(df_wo_null.corr(),annot= True,linewidths=0.5,fmt = ".1f",ax=ax)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.title('Correlation Map')
plt.show()

redundante Sachen weg:

In [ ]:
data_reduced_features = data_wo_null[['<your feature 1>', '<your feature 2>','...']] # die die drin bleiben
# überprüfen das ALLE werte drin sind (die die keine zahlen als werte haben)
data_reduced_features.head()

#### ANDERE MÖGLICHKEIT daten können auch gedropppt werden

In [ ]:
data_wo_null = df_wo_null.drop(['attr'], axis = 1) # 'attr' is no appropriate predictor

oder neue features, mit ranges

In [ ]:
df_wo_null['AgeRange'] = pd.cut(df_wo_null['Age'], bins=8) # summarize 'Age' in ranges, macht 8 ranges
df_wo_null[['AgeRange', 'Survived']].groupby(['AgeRange'], as_index=False).mean().sort_values(by='AgeRange', ascending=True)

In [ ]:
df_wo_null['AgeRange'] = pd.cut(df_wo_null['Age'], bins=8, labels = ['0-10','10-20','20-30','30-40','40-50','50-60','60-70', '70-80'])
# create a new column 'AgeRange'

In [ ]:
df_wo_null = df_wo_null.drop(['Age'], axis=1) # remove 'Age' since information is now contained in 'AgeRange'

#### outliers removen

In [ ]:
data_reduced_features.hist(figsize=(25,25), bins=50)

In [ ]:
q = data_reduced_features['total_rooms'].quantile(0.99)
data_reduced_features_2 = data_reduced_features[data_reduced_features['total_rooms']<q]

### so jetzt preparen für modeling

man kann dummies getten, aber geht auch ohne

In [ ]:
dummies = pd.get_dummies(data_reduced_features_2, drop_first=True)
dummies.head()

target und predictors bestimmen

In [ ]:
target = dummies['attr']
predictors = dummies.drop(['attr'], axis=1)

dann trainings und testdaten spliten

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.2, random_state=123) # 80-20 split into training and test data

predictors können gescalt werden (fit on training set and transform training and test set)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Jetzt kommen die Models

## Decision Tree

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

In [ ]:
confusion_matrix(y_test, tree.predict(X_test)) # yields count of true negatives, false positives, false negatives, true positives

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, tree.predict(X_test)).ravel() # check that tp, fp, tn, fn are not confused
print(tn, fp, fn, tp)

In [ ]:
print(classification_report(y_train, tree.predict(X_train))) # yields class-specific precision, recall and f1-score

In [ ]:
print(classification_report(y_test, tree.predict(X_test)))

wenn kacke mal feature importance anschauen

In [ ]:
list(zip(X_train.columns, tree.feature_importances_)) # lists features and their importance in predicting the target

und dann ruhig noch nen random forest hinterhauen

 bei bäumen immer versch. tiefen ausprobieren: max_depth=5

## Random Forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
confusion_matrix(y_test, rf.predict(X_test))

In [ ]:
print(classification_report(y_train, rf.predict(X_train)))

In [ ]:
print(classification_report(y_test, rf.predict(X_test)))

In [ ]:
list(zip(X_train.columns, rf.feature_importances_))

## Logistic Regression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
print(confusion_matrix(y_test, logreg.predict(X_test)))

In [ ]:
print(classification_report(y_test, logreg.predict(X_test)))

In [ ]:
print(classification_report(y_train, logreg.predict(X_train)))

schöne confusion matrix

In [ ]:
conf_mat = confusion_matrix(y_test, logreg.predict(X_test))
df_cm = pd.DataFrame(conf_mat, index=['0','1'], columns=['0', '1'],)
fig = plt.figure(figsize=[10,7])
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=14)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=14)
plt.ylabel('True label')
plt.xlabel('Predicted label')

## Linear Regression

In [ ]:
reg = LinearRegression()
reg.fit(X_train,y_train)

In [ ]:
print('training performance')
print(reg.score(X_train,y_train))
print('test performance')
print(reg.score(X_test,y_test))

In [ ]:
y_pred = reg.predict(X_test)
test = pd.DataFrame({'Predicted':y_pred,'Actual':y_test})
fig= plt.figure(figsize=(16,8))
test = test.reset_index()
test = test.drop(['index'],axis=1)
plt.plot(test[:50])
plt.legend(['Actual','Predicted'])
sns.jointplot(x='Actual',y='Predicted',data=test,kind='reg',);

# Deployment

In [ ]:
# import the Watson Machine Learning Python client library
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
# fill in your credentials
wml_credentials = {
    # your code
}

In [ ]:
# instantiate the client
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# review stored artifacts
wml_client.repository.list()

In [ ]:
# delete artifacts that are no longer needed (note that the number of stored artifacts in the free version is limited to 5)
wml_client.repository.delete("...GUID...")

In [ ]:
# fill in metadata for your deployment
metadata = {
        wml_client.repository.ModelMetaNames.NAME: 'Housing Prices Deployment',
        wml_client.repository.ModelMetaNames.DESCRIPTION: 'Test deployment for the california housing prices data set.',
        wml_client.repository.ModelMetaNames.AUTHOR_NAME: 'Your Name'
}

In [ ]:
# store the (scikit-learn) model in WML
stored_model = wml_client.repository.store_model(NAME, meta_props=metadata)

In [ ]:
# get the id of the stored model
published_model_uid = wml_client.repository.get_model_uid(stored_model)

In [ ]:
# create deployment and fetch scoring endpoint
created_deployment = wml_client.deployments.create(published_model_uid, name="Housing Prices Deployment")
scoring_endpoint = wml_client.deployments.get_scoring_url(created_deployment)

## Deployment Validation

In [ ]:
# review original data
original_data.head(2)

In [ ]:
# review predictors
predictors.head(2)

In [ ]:
# create scoring payload
scoring_values = predictors.iloc[0:2].to_numpy().tolist()
scoring_payload = {"values": scoring_values}
print(scoring_payload)

In [ ]:
# run prediction
predictions = wml_client.deployments.score(scoring_endpoint, scoring_payload)
print(predictions)

In [ ]:
# use the local model to make the same prediction in your notebook and compare the results
model.predict(predictors.iloc[0:2])